## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: H1
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# create a database to store optuna studies with sqlite backend
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [3]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [4]:
cell_line = CELL_LINES[2]
cell_line

'H1'

---

In [5]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [6]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [7]:
task = TASKS[0]

In [8]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [9]:
model=EmbraceNetMultimodal

In [10]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:32,326] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.10584814503981839
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 32
    lr: 4.366399279990093e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.30341574474062705
    weight_decay: 0.005075790764731148

===============> MODEL TESTING
AUPRC test score: 0.08181235767402777


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:33,060] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.09931226996424711
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 11
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 256
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.0002029719451244337
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.21521890983817527
    weight_decay: 0.003454362055483411

===============> MODEL TESTING
AUPRC test score: 0.08624962874303777


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:33,719] Using an existing study with name 'H1_active_E_vs_inactive_E_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1302299532117524
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    lr: 0.0004943337970140457
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.6959902847786185
    weight_decay: 0.0011828561736692122

===============> MODEL TESTING
AUPRC test score: 0.09541923649148885



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.08783


In [11]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [12]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [13]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [14]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [15]:
model=EmbraceNetMultimodal

In [16]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:36,371] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.3025317995726415
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 128
    EMBRACENET_n_units_l1: 64
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0003533773603162303
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.4615369060542335
    weight_decay: 0.0006454235737532866

===============> MODEL TESTING
AUPRC test score: 0.29145218167426606


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:36,768] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2615932584948042
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.0
    FFNN_n_layers: 2
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    lr: 0.026842343853951817
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.35360892078627604
    weight_decay: 0.0014650000353522048

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:37,100] Using an existing study with name 'H1_active_P_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.278506375420035


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.30162518603237853
  Params: 
    CNN_dropout_l0: 0.2
    CNN_kernel_size_l0: 11
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.0001157314508522964
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.8381915206447009
    weight_decay: 0.0005576570260230154

===============> MODEL TESTING
AUPRC test score: 0.31272556383596256



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.29423


In [17]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [18]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [19]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [20]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [21]:
model=EmbraceNetMultimodal

In [22]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:40,809] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_1' instead of creating a new one.


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5194680497918178
  Params: 
    CNN_dropout_l0: 0
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 11
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    EMBRACENET_dropout_l0: 0.2
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 32
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 0.005502817461910754
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.5779530307237474
    weight_decay: 0.00022581641525237922

===============> MODEL TESTING
AUPRC test score: 0.5964261704869845




/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:41,010] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.4844512770686714
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 15
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    EMBRACENET_n_units_l1: 256
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 32
    lr: 0.003938835812863552
    n_post_layers: 2
    optimizer: Adam
    selection_probabilities_FFNN: 0.869205014159509
    weight_decay: 0.00045573587180163243

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:41,252] Using an existing study with name 'H1_active_E_vs_active_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.


AUPRC test score: 0.4142285967190326


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5212409843096582
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 128
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.006878889009165519
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.46994610984970253
    weight_decay: 0.01305643040841192

===============> MODEL TESTING
AUPRC test score: 0.216984680981037



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.40921


In [23]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [24]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [25]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [26]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [27]:
model=EmbraceNetMultimodal

In [28]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:45,841] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5650554893490456
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 15
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 96
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 128
    FFNN_n_units_l2: 4
    lr: 0.01655764523137371
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.6416979453128385
    weight_decay: 0.00011713656180786807

===============> MODEL TESTING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:46,379] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2' instead of creating a new one.


AUPRC test score: 0.5583608876691454


>>> ITERATION N. 2

===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.591363906019812
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 32
    EMBRACENET_embracement_size: 1024
    FFNN_dropout_l0: 0.4
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 2.6549601142036214e-05
    n_post_layers: 0
    optimizer: Nadam
    selection_probabilities_FFNN: 0.7227169397398152
    weight_decay: 0.0006385391803407655

===============> MODEL TESTING
AUPRC test score: 0.5934009376839389


>>> ITERATION N. 3


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:46,778] Using an existing study with name 'H1_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5760002664977636
  Params: 
    CNN_dropout_l0: 0
    CNN_kernel_size_l0: 5
    CNN_n_layers: 1
    CNN_out_channels_l0: 64
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 128
    lr: 4.2337031432013654e-05
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.36154031934716324
    weight_decay: 0.003845718864247063

===============> MODEL TESTING
AUPRC test score: 0.6023799417685515



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.58471


In [29]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [30]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [31]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [32]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [33]:
model=EmbraceNetMultimodal

In [34]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:49,909] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2122103227118117
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.5
    CNN_dropout_l2: 0.5
    CNN_dropout_l3: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 11
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 64
    CNN_out_channels_l2: 128
    CNN_out_channels_l3: 256
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.0
    FFNN_n_layers: 1
    FFNN_n_units_l0: 64
    lr: 2.692148109170225e-05
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.09490670953375435
    weight_decay: 0.06417523893091548

===============> MODEL TESTING
AUPRC test score: 0.19490657013930532


>>> ITERATION N. 2


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:50,312] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2705892038912763
  Params: 
    CNN_dropout_l0: 0.3
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.5
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    EMBRACENET_dropout_l0: 0.5
    EMBRACENET_dropout_l1: 0.0
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.0
    FFNN_dropout_l2: 0.4
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 4
    lr: 0.00022909541299509693
    n_post_layers: 2
    optimizer: Nadam
    selection_probabilities_FFNN: 0.42263693660224033
    weight_decay: 0.00015608395251311087

===============> MODEL TESTING
AUPRC test 

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-26 09:39:50,743] Using an existing study with name 'H1_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2_3' instead of creating a new one.



===============> HYPERPARAMETERS TUNING
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.2487836279707956
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_n_layers: 3
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 256
    EMBRACENET_dropout_l0: 0.3
    EMBRACENET_embracement_size: 512
    EMBRACENET_n_units_l0: 256
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.5
    FFNN_n_layers: 3
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 4
    lr: 0.001676879275274851
    n_post_layers: 1
    optimizer: Nadam
    selection_probabilities_FFNN: 0.1852152964098721
    weight_decay: 0.0002164412865447057

===============> MODEL TESTING
AUPRC test score: 0.2426846699049893



3-FOLD CROSS-VALIDATION AUPRC TEST SCOR

In [35]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [36]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

## NO FREEZED LAYERS

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## FREEZED LAYERS + BO + no augmentation

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

In [ ]:
plot_scores(cells=cell_line)